## Creating the CNN model

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import pip
import numpy as np
import scipy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import ImageFile,Image
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import PIL
from PIL import ImageTk, ImageDraw, Image
from tkinter import *
import os


In [21]:
classifier = Sequential()

classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = (28,28,1)))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(.2))

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu',padding='same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(.2))

classifier.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu',padding='same'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#classifier.add(Dropout(.2))


classifier.add(Flatten())

classifier.add(Dense(units = 64, activation = 'relu'))
#classifier.add(Dropout(.2))
classifier.add(Dense(128,activation="relu"))

classifier.add(Dense(units = 50, activation = 'softmax'))

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


## Fitting the CNN model to the Dataset

In [43]:
train_folder = "Dataset/Dataset/Train" 
test_folder = "Dataset/Dataset/Test"
dimensions = (50, 50)

In [38]:
def make_greyscale_white_bg(im, r, b, g):

    im = im.convert('RGBA')   # Convert to RGBA


    data = np.array(im)   # "data" is a height x width x 4 numpy array
    red, green, blue, alpha = data.T # Temporarily unpack the bands for readability

    # Replace grey with white... (leaves alpha values alone...)
    grey_areas = (red == r) & (blue == b) & (green == g)
    data[..., :-1][grey_areas.T] = (255, 255, 255) # Transpose back needed

    im2 = Image.fromarray(data)
    im2 = im2.convert('L')   # convert to greyscale image


    return im2

In [39]:
def invert_colors(im):

    im = im.convert('RGBA')   # Convert to RGBA
    data = np.array(im)   # "data" is a height x width x 4 numpy array
    red, green, blue, alpha = data.T # Temporarily unpack the bands for readability


    # Replace black with red temporarily... (leaves alpha values alone...)
    black_areas = (red == 0) & (blue == 0) & (green == 0)
    data[..., :-1][black_areas.T] = (255, 0, 0) # Transpose back needed

    # Replace white areas with black
    white_areas = (red == 255) & (blue == 255) & (green == 255)
    data[..., :-1][white_areas.T] = (0, 0, 0) # Transpose back needed

    # Replace red areas (originally white) with black
    red_areas = (red == 255) & (blue == 0) & (green == 0)
    data[..., :-1][red_areas.T] = (255, 255, 255) # Transpose back needed

    im2 = Image.fromarray(data)
    im2 = im2.convert('L')   # convert to greyscale image


    return im2

In [41]:
def process_images(folder):

    classes = [os.path.join(folder, d) for d in sorted(os.listdir(folder))]  # get list of all sub-folders in folder
    img_cnt = 0

    for class_x in classes:

        if os.path.isdir(class_x):

            # get paths to all the images in this folder
            images = [os.path.join(class_x, i) for i in sorted(os.listdir(class_x)) if i != '.DS_Store']

            for image in images:

                img_cnt = img_cnt + 1

                if(img_cnt % 1000 == 0):                # show progress
                    print("Processed %s images" % str(img_cnt))

                try:
                    im = Image.open(image)
                except OSError:
                    continue
                print(image)
                im = im.resize(dimensions)   # resize image according to dimensions set

                im = make_greyscale_white_bg(im, 127, 127, 127) # turn grey background (if any) to white, and                                                 # convert into greyscale image with 1 channel
                  # overwrite previous image file with new image
                im = invert_colors(im)
                im.save(image)
    print("Finished processing images, images found = ")
    print(img_cnt)
process_images(train_folder)

Dataset/Dataset/Train\172\bcc000000.bmp
Dataset/Dataset/Train\172\bcc000001.bmp
Dataset/Dataset/Train\172\bcc000002.bmp
Dataset/Dataset/Train\172\bcc000003.bmp
Dataset/Dataset/Train\172\bcc000004.bmp
Dataset/Dataset/Train\172\bcc000005.bmp
Dataset/Dataset/Train\172\bcc000006.bmp
Dataset/Dataset/Train\172\bcc000007.bmp
Dataset/Dataset/Train\172\bcc000008.bmp
Dataset/Dataset/Train\172\bcc000009.bmp
Dataset/Dataset/Train\172\bcc000010.bmp
Dataset/Dataset/Train\172\bcc000011.bmp
Dataset/Dataset/Train\172\bcc000012.bmp
Dataset/Dataset/Train\172\bcc000013.bmp
Dataset/Dataset/Train\172\bcc000014.bmp
Dataset/Dataset/Train\172\bcc000015.bmp
Dataset/Dataset/Train\172\bcc000016.bmp
Dataset/Dataset/Train\172\bcc000017.bmp
Dataset/Dataset/Train\172\bcc000018.bmp
Dataset/Dataset/Train\172\bcc000019.bmp
Dataset/Dataset/Train\172\bcc000020.bmp
Dataset/Dataset/Train\172\bcc000021.bmp
Dataset/Dataset/Train\172\bcc000022.bmp
Dataset/Dataset/Train\172\bcc000023.bmp
Dataset/Dataset/Train\172\bcc000024.bmp


In [9]:
ImageFile.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = .2, rotation_range = 25)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Dataset\Dataset\Train', target_size = (28, 28), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")
test_set = test_datagen.flow_from_directory('Dataset\Dataset\Test', target_size = (28, 28), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")


Found 12000 images belonging to 50 classes.
Found 3000 images belonging to 50 classes.


In [46]:
len(training_set)


12000

In [47]:
len(test_set)

3000

In [68]:
tf.keras.backend.clear_session()

In [81]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                  patience=5, min_lr=0.0001)
classifier.fit(training_set ,steps_per_epoch = 12000, epochs = 5,
                        validation_data = test_set, validation_steps = 3000,callbacks=[reduce_lr])

Epoch 1/5
12000/12000 [==============================] - 56s 5ms/step - loss: 0.3209 - accuracy: 0.9014 - val_loss: 0.6733 - val_accuracy: 0.8720 - lr: 2.0000e-04
Epoch 2/5
12000/12000 [==============================] - 56s 5ms/step - loss: 0.3321 - accuracy: 0.9004 - val_loss: 0.6966 - val_accuracy: 0.8720 - lr: 2.0000e-04
Epoch 3/5
12000/12000 [==============================] - 56s 5ms/step - loss: 0.3254 - accuracy: 0.8971 - val_loss: 0.6801 - val_accuracy: 0.8743 - lr: 2.0000e-04
Epoch 4/5
12000/12000 [==============================] - 57s 5ms/step - loss: 0.3282 - accuracy: 0.9018 - val_loss: 0.6603 - val_accuracy: 0.8763 - lr: 2.0000e-04
Epoch 5/5
12000/12000 [==============================] - 57s 5ms/step - loss: 0.3001 - accuracy: 0.9044 - val_loss: 0.6454 - val_accuracy: 0.8717 - lr: 2.0000e-04


In [20]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 6, 6, 128)         7

## Saving the model

In [37]:
classifier_json = classifier.to_json()

with open("CNN_BanglaHandWrittenCharacterRecognition.json", "w") as json_file:
    json_file.write(classifier_json)
    
classifier.save_weights("CNN_BanglaHandWrittenCharacterRecognition.h5")
print('Saved model to disk')

NameError: name 'classifier' is not defined

## Creating a graphical user interface to draw the character

In [22]:
import PIL
from PIL import ImageDraw,Image
from tkinter import *


def create_new_image():
    width = 256
    height = 256
    
    center = height // 2
    white = (255, 255, 255)
    green = (0, 128, 0)
    
    def save():
        filename = 'C:\\Users\Saunak\Documents\GitHub\RegionalCharacterDetectionUsingCNN\Dataset\Dataset\SinglePrediction\image.jpg'
        image.save(filename)
        
    def paint(event):
        x1, y1 = (event.x - 1), (event.y - 1)
        x2, y2 = (event.x + 1), (event.y + 1)
        cv.create_oval(x1, y1, x2, y2, fill = 'white', width = 30)
        draw.line([x1, y1, x2, y2], fill = 'white', width = 30)
        
    root = Tk()
    
    cv = Canvas(root, width = width, height = height, bg = 'black')
    cv.pack()
    
    image = PIL.Image.new('RGB', (width, height), white)
    draw = ImageDraw.Draw(image)
    
    cv.pack(expand = YES, fill = BOTH)
    cv.bind("<B1-Motion>", paint)
    
    button = Button(text = 'Save', command = save)
    button.pack()
    
    root.mainloop()

In [12]:
def determine_character(res):
    if res == 0:
        print('prediction : অ')
    elif res == 1:
        print('prediction : আ')
    elif res == 2:
        print('prediction : ই')
    elif res == 3:
        print('prediction : ঈ')
    elif res == 4:
        print('prediction : উ')
    elif res == 5:
        print('prediction : ঊ')
    elif res == 6:
        print('prediction : ঋ')
    elif res == 7:
        print('prediction : এ')
    elif res == 8:
        print('prediction : ঐ')
    elif res == 9:
        print('prediction : ও')
    elif res == 10:
        print('prediction : ঔ')
    elif res == 11:
        print('prediction : ক')
    elif res == 12:
        print('prediction : খ')
    elif res == 13:
        print('prediction : গ')
    elif res == 14:
        print('prediction : ঘ')
    elif res == 15:
        print('prediction : ঙ')
    elif res == 16:
        print('prediction : চ')
    elif res == 17:
        print('prediction : ছ')
    elif res == 18:
        print('prediction : জ')
    elif res == 19:
        print('prediction : ঝ')
    elif res == 20:
        print('prediction : ঞ')
    elif res == 21:
        print('prediction : ট')
    elif res == 22:
        print('prediction : ঠ')
    elif res == 23:
        print('prediction : ড')
    elif res == 24:
        print('prediction : ঢ')
    elif res == 25:
        print('prediction : ণ')
    elif res == 26:
        print('prediction : ত')
    elif res == 27:
        print('prediction : থ')
    elif res == 28:
        print('prediction : দ')
    elif res == 29:
        print('prediction : ধ')
    elif res == 30:
        print('prediction : ন')
    elif res == 31:
        print('prediction : প')
    elif res == 32:
        print('prediction : ফ')
    elif res == 33:
        print('prediction : ব')
    elif res == 34:
        print('prediction : ভ')
    elif res == 35:
        print('prediction : ম')
    elif res == 36:
        print('prediction : য')
    elif res == 37:
        print('prediction : র')
    elif res == 38:
        print('prediction : ল')
    elif res == 39:
        print('prediction : শ')
    elif res == 40:
        print('prediction : ষ')
    elif res == 41:
        print('prediction : স')
    elif res == 42:
        print('prediction : হ')
    elif res == 43:
        print('prediction : ড়')
    elif res == 44:
        print('prediction : ঢ়')
    elif res == 45:
        print('prediction : য়')
    elif res == 46:
        print('prediction : ৎ')
    elif res == 47:
        print('prediction : ং')
    elif res == 48:
        print('prediction : ঃ')
    else:
        print('prediction : ঁ')

In [23]:


classifier.load_weights('CNN_BanglaHandWrittenCharacterRecognition.h5')

In [29]:


def single_prediction(test_img):
    test_img_arr = keras.utils.img_to_array(test_img)
    test_img_arr = np.expand_dims(test_img_arr, axis = 0)
    prediction = classifier.predict(test_img_arr)
    result = np.argmax(prediction, axis = 1)
    print(result)
    determine_character(result)

In [17]:
def delete_created_image():
    os.remove('C:\\Users\Saunak\Documents\GitHub\RegionalCharacterDetectionUsingCNN\Dataset\Dataset\SinglePrediction\image.jpg')

In [27]:

def draw_n_guess_the_character():
    create_new_image()
    test_img = keras.utils.load_img('C:\\Users\Saunak\Documents\GitHub\RegionalCharacterDetectionUsingCNN\Dataset\Dataset\SinglePrediction\image.bmp', target_size = (28, 28, 1))
    single_prediction(test_img)
    plt.imshow(test_img)
    delete_created_image()

In [30]:
draw_n_guess_the_character()


ValueError: in user code:

    File "c:\Users\Saunak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\Saunak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Saunak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\Saunak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1983, in predict_step
        return self(x, training=False)
    File "c:\Users\Saunak\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\